In [1]:
import os

os.environ["COHERE_API_KEY"]='QdWMbWNWgMbj4UpVyM5lSR5nFr1PIL0SoBrZ4XP3'
from langchain.llms import Cohere
llm = Cohere()

C:\Users\KABIR\AppData\Local\Temp\ipykernel_21720\657638532.py:5: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere()


In [2]:
from langchain.document_loaders import PyPDFLoader

filename='EMPLOYEE_AGREEMENT.pdf'

loaded= PyPDFLoader(filename)
pages=loaded.load()
print(len(pages))

13


# split into chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)
chunks = text_spliter.split_documents(pages)

In [5]:
chunks[0]

Document(metadata={'source': 'EMPLOYEE_AGREEMENT.pdf', 'page': 0}, page_content='EX-10.1 3 smtp_ex10z1.htm EMPLOYEE AGREEMENT\nEXHIBIT 10.1\nEMPLOYEE AGREEMENT\nTHIS EMPLOYEE AGREEMENT made as of September___, 2014, by and between SharpSpring,\nInc., a Delaware corporation (the “Company”), whose principal place of business is at 802 NW 5th Avenue,')

In [6]:
len(chunks)

133

# create embeddding

In [8]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(user_agent="langchain")

C:\Users\KABIR\AppData\Local\Temp\ipykernel_21720\1239584916.py:2: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings = CohereEmbeddings(user_agent="langchain")


# vectore store

In [11]:
from langchain.vectorstores import FAISS

In [12]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [15]:
retriever = vectorstore.as_retriever()

In [16]:
retriever

VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C19B525610>, search_kwargs={})

In [17]:
query = "What are the key clauses in the employee agreement?"
response = retriver.get_relevant_documents(query)
print(response[0].page_content)

C:\Users\KABIR\AppData\Local\Temp\ipykernel_21720\3254707316.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retriver.get_relevant_documents(query)


agreed upon by Employee.


In [19]:
query = "who is PM"
response = retriver.get_relevant_documents(query)
print(response[0].page_content)

terminated by either party at any time in accordance with the provisions of Article 7.
ARTICLE 2
Duties
2.1. Position and Duties. The Company agrees to employ Employee to act as its President. Employee


In [20]:
from langchain.chains import ConversationalRetrievalChain

In [21]:
from langchain.memory import ConversationBufferMemory

In [29]:
conversational_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory= ConversationBufferMemory(memory_key='chat_history', return_messages=True)
)

In [30]:
res = conversational_chain({'question':"What are the key clauses in the employee agreement?"})

In [33]:
res

{'question': 'What are the key clauses in the employee agreement?',
 'chat_history': [HumanMessage(content='What are the key clauses in the employee agreement?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. ", additional_kwargs={}, response_metadata={})],
 'answer': " The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. "}

In [36]:
m = res.get('chat_history', [])

In [38]:
m

[HumanMessage(content='What are the key clauses in the employee agreement?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=" The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. ", additional_kwargs={}, response_metadata={})]

In [41]:
for i, message in enumerate(m):
    print(message.content)

What are the key clauses in the employee agreement?
 The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. 


In [42]:
m[0].content

'What are the key clauses in the employee agreement?'

In [44]:
from langchain.chains import RetrievalQA




# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Create Retrieval Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Ask a question
query = "What are the key clauses in the employee agreement?"
response = qa_chain.run(query)

print("Answer:", response)

C:\Users\KABIR\AppData\Local\Temp\ipykernel_21720\1738161444.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Answer:  The key clauses in this employee agreement are the terms and conditions under which the employee will be employed. These include the agreed-upon wages and the length of employment. It also includes the context of their relationship, describing the employer and employee relationship. 


In [45]:
res = qa_chain.run("What is machine Learning")

In [46]:
print(res)

 Machine learning (ML) is a subset of artificial intelligence (AI) and refers to the ability of certain computer systems to learn and improve from experience automatically without being explicitly programmed. It focuses on developing algorithms and models that can absorb data, identify patterns and make decisions or predictions based on that data. 

ML can be applied to various fields, including marketing automation, customer relationship management, sales automation, and email delivery to enhance and optimize these processes. For instance, ML can be used to analyze customer behavior, predict future behaviors and trends accurately, and personalize marketing strategies. 

Is there anything specific you would like to know about Machine Learning, or would you like me to provide more general information on the topic.?



In [47]:
res = conversational_chain({'question':"What is machine learning?"})

In [48]:
res

{'question': 'What is machine learning?',
 'chat_history': [HumanMessage(content='What are the key clauses in the employee agreement?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. ", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" I don't have access to any specific information about the context you mentioned, and this is not a topic I am trained on. That said, generally speaking, machine learning (ML) is a branch of computer science that focuses on the use of data and algorithms to imitate the way that humans learn,

In [51]:
query = "Where is America?"
retrieved_docs = retriever.get_relevant_documents(query)
print("Retrieved Docs:", [doc.page_content for doc in retrieved_docs])

Retrieved Docs: ['West Palm Beach, Florida 33401', 'ARTICLE 3\nPlace of Employment\n3.1. Place of Employment.   Employee shall perform his duties under this Employee Agreement at\n802 NW 5th Avenue, Suite 100, Gainesville FL 32601.\nARTICLE 4\nCompensation of Employee', 'world based upon the fact that Company and /or the Affiliates currently offer their services in approximately\n130 countries and Company and /or the Affiliates plan to launch products and']


In [52]:
res = qa_chain.run(query)

In [53]:
res

" America is a large continent that is also referred to as the United States of America, which is a country in the continent's northern hemisphere. The geographic coordinates of the continental unity of the Northern American continent are between the Arctic and the Atlantic oceans, the Caribbean Sea and the Pacific Ocean. \n\nThe capital of the country is Washington DC, where the central government is located. However, the most famous city in the country is New York, which is located in the state of New York, on the northeastern border of the country. Interestingly, the city of New York, also known as NYC, is the most densely populated city in the country, with over 26,000 people per square mile. \n\nThe employee agreement you provided originates from Florida, located on the southeastern coast, which borders the Atlantic Ocean and the Gulf of Mexico and is the longest coastline among all states. "

In [54]:
res = conversational_chain({'question':query})

In [55]:
res

{'question': 'Where is America?',
 'chat_history': [HumanMessage(content='What are the key clauses in the employee agreement?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" The key clauses in this employee agreement are the terms and conditions detailing the employee's role, the agreed-upon payments for their work, the length of their employment, and the date employment is to begin. These are all standard clauses in an employment contract designed to protect the interests of both the employer and employee. ", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=" I don't have access to any specific information about the context you mentioned, and this is not a topic I am trained on. That said, generally speaking, machine learning (ML) is a branch of computer science that focuses on the use of data and algorithms to imitate the way that humans learn, gradual